In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
import textwrap

# 1. "Bättre" leksaksdata – längre text
raw_text = """
Du är en liten språkmodell som tränas på text.
Du kan lära dig mönster i sekvenser av tecken.
Det här är bara exempeltext, men den är lite längre än tidigare.
Vi kan lägga till fler meningar för att göra träningen mer intressant.
Språkmodellen ska försöka förutsäga nästa tecken i texten.
""".lower()

# Ta bort leading indentation
text = textwrap.dedent(raw_text)

print("Antal tecken i text:", len(text))
print(text[:200])


Antal tecken i text: 290

du är en liten språkmodell som tränas på text.
du kan lära dig mönster i sekvenser av tecken.
det här är bara exempeltext, men den är lite längre än tidigare.
vi kan lägga till fler meningar för att 


In [2]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("Antal unika tecken:", vocab_size)
print(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

def encode(s: str):
    return [stoi[c] for c in s]

def decode(ids):
    return "".join(itos[i] for i in ids)


Antal unika tecken: 28
['\n', ' ', ',', '.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'x', 'ä', 'å', 'ö']


In [3]:
data = torch.tensor(encode(text), dtype=torch.long)

block_size = 64   # hur långa sekvenser modellen får se
print("Total längd på data:", len(data))

def get_batch(batch_size=32, split="train"):
    # För enkelhet: inga valideringssplit nu, bara slumpbatchar
    ix = torch.randint(0, len(data) - block_size - 1, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


Total längd på data: 290


In [4]:
class MultiHeadSelfAttention(nn.Module):
    def __init__(self, d_model, num_heads, block_size):
        super().__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_head = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model, bias=False)
        self.W_k = nn.Linear(d_model, d_model, bias=False)
        self.W_v = nn.Linear(d_model, d_model, bias=False)
        self.W_o = nn.Linear(d_model, d_model, bias=False)

        # Causal mask (T, T)
        mask = torch.tril(torch.ones(block_size, block_size))
        self.register_buffer("mask", mask)

    def forward(self, x):
        B, T, C = x.shape

        # 1) Linjära projektioner
        Q = self.W_q(x)  # (B, T, C)
        K = self.W_k(x)
        V = self.W_v(x)

        # 2) Dela upp i heads: (B, T, H, d_head) -> (B, H, T, d_head)
        Q = Q.view(B, T, self.num_heads, self.d_head).transpose(1, 2)
        K = K.view(B, T, self.num_heads, self.d_head).transpose(1, 2)
        V = V.view(B, T, self.num_heads, self.d_head).transpose(1, 2)

        # 3) Scaled dot-product attention
        # scores: (B, H, T, T)
        scores = Q @ K.transpose(-2, -1) / math.sqrt(self.d_head)

        # 4) Causal mask
        mask = self.mask[:T, :T]  # (T, T)
        scores = scores.masked_fill(mask == 0, float("-inf"))

        # 5) Softmax över "vem tittar jag på"
        A = F.softmax(scores, dim=-1)  # (B, H, T, T)

        # 6) Vägda summor
        out = A @ V  # (B, H, T, d_head)

        # 7) Tillbaka till (B, T, C)
        out = out.transpose(1, 2).contiguous().view(B, T, C)

        # 8) Ut-projektion
        out = self.W_o(out)  # (B, T, C)
        return out


In [5]:
class TransformerBlock(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, block_size):
        super().__init__()
        self.attn = MultiHeadSelfAttention(d_model, num_heads, block_size)
        self.ln1 = nn.LayerNorm(d_model)
        self.ff = nn.Sequential(
            nn.Linear(d_model, d_ff),
            nn.ReLU(),
            nn.Linear(d_ff, d_model),
        )
        self.ln2 = nn.LayerNorm(d_model)

    def forward(self, x):
        # Self-attention del
        x = x + self.attn(self.ln1(x))  # pre-norm variant
        # FFN-del
        x = x + self.ff(self.ln2(x))
        return x


In [6]:
class MiniGPTChar(nn.Module):
    def __init__(self, vocab_size, block_size, d_model=128, num_heads=4, d_ff=256, num_layers=3):
        super().__init__()
        self.block_size = block_size
        self.d_model = d_model

        self.token_emb = nn.Embedding(vocab_size, d_model)
        self.pos_emb = nn.Embedding(block_size, d_model)

        self.blocks = nn.ModuleList([
            TransformerBlock(d_model, num_heads, d_ff, block_size)
            for _ in range(num_layers)
        ])

        self.ln_f = nn.LayerNorm(d_model)
        self.head = nn.Linear(d_model, vocab_size)

    def forward(self, idx, targets=None):
        # idx: (B, T)
        B, T = idx.shape
        assert T <= self.block_size

        pos = torch.arange(T, device=idx.device)
        tok_emb = self.token_emb(idx)          # (B, T, d_model)
        pos_emb = self.pos_emb(pos)[None, :, :]  # (1, T, d_model)
        x = tok_emb + pos_emb                  # (B, T, d_model)

        for block in self.blocks:
            x = block(x)

        x = self.ln_f(x)
        logits = self.head(x)                  # (B, T, vocab_size)

        loss = None
        if targets is not None:
            B, T, C = logits.shape
            logits_flat = logits.view(B*T, C)
            targets_flat = targets.view(B*T)
            loss = F.cross_entropy(logits_flat, targets_flat)

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens=100):
        # idx: (B, T_start)
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -self.block_size:]
            logits, _ = self(idx_cond)
            logits_last = logits[:, -1, :]  # (B, vocab_size)
            probs = F.softmax(logits_last, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1)  # (B, 1)
            idx = torch.cat([idx, next_token], dim=1)
        return idx


In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

model = MiniGPTChar(
    vocab_size=vocab_size,
    block_size=block_size,
    d_model=128,
    num_heads=4,
    d_ff=256,
    num_layers=3,
).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

for step in range(2000):
    model.train()
    xb, yb = get_batch(batch_size=32)
    xb, yb = xb.to(device), yb.to(device)

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if step % 200 == 0:
        print(f"Steg {step}, loss {loss.item():.4f}")


Device: cpu
Steg 0, loss 3.6161
Steg 200, loss 0.7941
Steg 400, loss 0.1129
Steg 600, loss 0.0623
Steg 800, loss 0.0587
Steg 1000, loss 0.0491
Steg 1200, loss 0.0466
Steg 1400, loss 0.0415
Steg 1600, loss 0.0453
Steg 1800, loss 0.0445


In [12]:
model.eval()
start_text = "språk"
start_ids = torch.tensor([encode(start_text)], dtype=torch.long).to(device)

gen_ids = model.generate(start_ids, max_new_tokens=200)
print(decode(gen_ids[0].tolist()))


språkmodell som tränas på text.
du kan lära dig mönster i sekvenser av tecken.
det här är bara exempeltext, men den är lite längre än tidigare.
vi kan lägga till fler meningar för att göra träningen mer in
